# Population Run at Scale

- **Goal:** Execute a 1000-subject population simulation using the in-memory adapter.
- **Data:** `reference/models/standard/midazolam_adult.pkml`.
- **Targets:** Wall-clock ≤ 12 minutes; aggregates persisted to `var/population-results/`.


## Steps

1. Ensure dependencies are installed (`pip install -e '.[dev]'`).
2. Run the cell to submit a population job and retrieve aggregates / chunk metadata.
3. Inspect the output JSON and verify claim-check artefacts under `var/population-results/`.


In [ ]:
import json
from pathlib import Path

from mcp import session_registry
from mcp.tools.get_population_results import GetPopulationResultsRequest, get_population_results
from mcp.tools.run_population_simulation import RunPopulationSimulationRequest, run_population_simulation
from mcp_bridge.adapter.interface import AdapterConfig
from mcp_bridge.adapter.mock import InMemoryAdapter
from mcp_bridge.services.job_service import JobService
from mcp_bridge.storage.population_store import PopulationResultStore

model_path = Path("reference/models/standard/midazolam_adult.pkml").resolve()
store = PopulationResultStore(Path("var/usecases/population-store"))
adapter = InMemoryAdapter(AdapterConfig(model_search_paths=[str(model_path.parent)]), population_store=store)
adapter.init()
job_service = JobService(max_workers=4, default_timeout=600.0, max_retries=1)

try:
    request = RunPopulationSimulationRequest(
        modelPath=str(model_path),
        simulationId="usecase-pop",
        cohort={"size": 1000, "sampling": "latin-hypercube"},
        metadata={"replicates": 10, "note": "Use-case pack demo"},
    )
    response = run_population_simulation(adapter, job_service, request)
    job = job_service.wait_for_completion(response.job_id, timeout=600.0)
    results = get_population_results(adapter, GetPopulationResultsRequest(resultsId=job.result_id))
    print(json.dumps(results.model_dump(mode="json"), indent=2))
finally:
    job_service.shutdown()
    adapter.shutdown()
    session_registry.clear()


## Reference Metrics

The aggregates reported for the mock adapter scale linearly with cohort size (e.g.
`meanCmax ≈ 1.0 + 0.01 × cohort_size`). When running against a real ospsuite backend,
compare the output to benchmarks captured in `docs/mcp-bridge/performance-plan.md`.
